In [1]:
def sell_order_past_hour(order_time_conv_pst):
    curr_time = datetime.now()
    curr_time_form = curr_time.strftime("%Y-%m-%d %H:%M:%S")
    curr_time_form = str(curr_time_form)
    
    curr_time_conv = datetime.strptime(curr_time_form, "%Y-%m-%d %H:%M:%S")

    diff = curr_time_conv-order_time_conv_pst
    
    if diff.total_seconds() > 3600:
        return True
    else:
        return False

In [2]:
def get_PST_time(order_time):
    order_time_form = order_time.replace('T', " ")
    order_time_form = order_time_form.replace ('Z', "")
    order_time_conv = datetime.strptime(order_time_form, "%Y-%m-%d %H:%M:%S.%f")
    order_time_conv_pst = order_time_conv - timedelta(hours=7) 
    return order_time_conv_pst

In [3]:
def get_sell_limit_orders():
    sell_limit_orders = []
    #get all stock orders
    orders_list = rs.get_all_stock_orders()
    for orders in orders_list:
        if orders['side'] == 'sell' and orders['state'] == 'confirmed':
            #order_id and created_time
            #print(orders)
            #print('\n')
            #instrument needed
            instrument = orders['instrument']
            ticker = rs.stocks.get_symbol_by_url(instrument)            
            append_val = [orders['id'], orders['created_at'], ticker]
            sell_limit_orders.append(append_val)
    return sell_limit_orders

In [6]:
def remove_stale_sell_orders():
    #get orderIDs for all sell limit order
    sell_limit_orders = get_sell_limit_orders()
    #if sell limit order has been in place for more than an hour
    for orders in sell_limit_orders:
        order_time_conv_pst = get_PST_time(orders[1])
        if sell_order_past_hour(order_time_conv_pst):
            #cancel the sell limit order
            rs.orders.cancel_stock_order(orders[0])
            time.sleep(5)

            #place an immediate sell order for that share
            sell_order = rs.orders.order_sell_market(orders[2], 1, timeInForce='gtc', extendedHours=False) 